# Introduction
Here we will be building a GPT-like model from scratch based on the two papers [Attention is All You Need](https://arxiv.org/abs/1706.03762), which proposed the **transformer** architecture, and [GPT-3](https://arxiv.org/abs/2005.14165). GPT is a language model that given an input simple predicts the next word.

# Libraries

In [1]:
%matplotlib inline
%config IPCompleter.use_jedi=False

In [2]:
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Data
We import the tiny-Shakespeare dataset and process it such that it can be used for creating a model that can create Shakespeare texts.

### Reading and Inspecting

In [3]:
with open("../../data/tinyshakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [4]:
print("Number of Characters: ", len(text))

Number of Characters:  1115394


In [5]:
# First 300 characters
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [6]:
# All unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Vocab Size: {vocab_size}")
print(f"Vocab Chars: {''.join(chars):}")

Vocab Size: 65
Vocab Chars: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


### Building Vocabulary and Encoder/Decoder
We just use a character-level tokenizer here, but in practice people e.g. OpenAI uses something else e.g. the **BPE** tokenizer:

In [7]:
# OpenAI encoder/decoder
import tiktoken
enc = tiktoken.get_encoding("gpt2")
enc.decode(enc.encode("hello world")) == "hello world"

True

We will be using a simple tokenizer that uses characters rather than word-chunks to make things easier to understand.

In [8]:
# Building the vocabulary
ctoi = {s:i for i,s in enumerate(chars)}; print(ctoi)
itoc = {i:s for s,i in ctoi.items()}; print(itoc)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [9]:
# Building encoder/decoder
encode = lambda s: [ctoi[c] for c in s]
decode = lambda l: ''.join([itoc[i] for i in l])

In [10]:
# Testing encoder/decoder
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


### Tokenizing
Using our simple tokenizer we tokenize the entire dataset.

In [11]:
# Tokenizing dataset
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:50])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56])


### Train/Valid Split

In [12]:
n = int(0.9*len(data))
data_train = data[:n]
data_valid = data[n:]

### Creating dataset
When builing out model, we would like it to be able to generate text from as little a context as one character, but still up to a context of size **block_size**.

In [13]:
# Example of a sample
block_size = 8
x = data_train[:block_size]
y = data_train[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} target is: {target}")

when input is tensor([18]) target is: 47
when input is tensor([18, 47]) target is: 56
when input is tensor([18, 47, 56]) target is: 57
when input is tensor([18, 47, 56, 57]) target is: 58
when input is tensor([18, 47, 56, 57, 58]) target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is: 58


We create a function for getting random batches from the data.

In [14]:
def get_batch(split: str, batch_size: int, block_size: int) -> torch.tensor:
    """
    Description:
        Generates a batch of data of inputs x and targets y.
    Inputs:
        split: test or valid split
        batch_size: How many independent sequences will be processed in parallel
        block_size: Maximum context length
    Outputs:
        x, y: a tuple with xs and ys
    """
    data = data_train if split == 'train' else data_valid
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [15]:
# Testing function
batch_size = 4 
block_size = 8 

xb, yb = get_batch('train', batch_size, block_size)
print('inputs:')
print(xb.shape)
print(xb)
print('outputs:')
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} target is: {target}")

inputs:
torch.Size([4, 8])
tensor([[51, 63,  1, 52, 53, 58, 43, 11],
        [ 6,  0, 32, 53,  1, 50, 39, 63],
        [ 1, 61, 43, 50, 41, 53, 51, 43],
        [56, 43, 58, 58, 63,  1, 58, 39]])
outputs:
torch.Size([4, 8])
tensor([[63,  1, 52, 53, 58, 43, 11,  0],
        [ 0, 32, 53,  1, 50, 39, 63,  1],
        [61, 43, 50, 41, 53, 51, 43,  2],
        [43, 58, 58, 63,  1, 58, 39, 50]])
when input is [51] target is: 63
when input is [51, 63] target is: 1
when input is [51, 63, 1] target is: 52
when input is [51, 63, 1, 52] target is: 53
when input is [51, 63, 1, 52, 53] target is: 58
when input is [51, 63, 1, 52, 53, 58] target is: 43
when input is [51, 63, 1, 52, 53, 58, 43] target is: 11
when input is [51, 63, 1, 52, 53, 58, 43, 11] target is: 0
when input is [6] target is: 0
when input is [6, 0] target is: 32
when input is [6, 0, 32] target is: 53
when input is [6, 0, 32, 53] target is: 1
when input is [6, 0, 32, 53, 1] target is: 50
when input is [6, 0, 32, 53, 1, 50] target is:

# Neural Network: Part I
Now we will start feeding the data into a neural network. We will just start by using the bigram model similar to the one we build previously.

In [16]:
# A batch
print(xb)
print(yb)

tensor([[51, 63,  1, 52, 53, 58, 43, 11],
        [ 6,  0, 32, 53,  1, 50, 39, 63],
        [ 1, 61, 43, 50, 41, 53, 51, 43],
        [56, 43, 58, 58, 63,  1, 58, 39]])
tensor([[63,  1, 52, 53, 58, 43, 11,  0],
        [ 0, 32, 53,  1, 50, 39, 63,  1],
        [61, 43, 50, 41, 53, 51, 43,  2],
        [43, 58, 58, 63,  1, 58, 39, 50]])


In [17]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        """ Creating Embedding Table """
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx: torch.tensor, targets: torch.tensor=None) -> tuple:
        """ Calculating the Loss """
        logits = self.token_embedding_table(idx) # (BATCH, TIME, CHANNEL)
        if targets is None:
            loss = None
        else:
            # Loss function takes (BATCH, CHANNEL, TIME) so we rearrange
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx: torch.tensor, max_new_tokens: int) -> torch.tensor:
        """ Generates Tokens Using a Sliding Window """
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [18]:
# Expected loss 
-math.log(1/vocab_size)

4.174387269895637

In [19]:
# Creating model and calculating loss
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.7054, grad_fn=<NllLossBackward0>)


In [20]:
# Generating some text
model_input = torch.zeros((1,1), dtype=torch.long) # Input token 0, which is \n
model_output = model.generate(model_input, max_new_tokens=100)[0].tolist()
print(f"Generated Text: \n {decode(model_output)}")

Generated Text: 
 
A,YPtg.C:j$.E!.?$g d!L,YdPHUeaiUD? cimfEc?'?X
fOYd,-Aythl.'JicgrttoXLvlQbCdetBOPPd&
Pas?NHdzXTnnMM:m


# Training a Model: Part I
Here we are going to use the Adam optimizer instead or stochastic gratient descent, which we used earlier. The optimizer is basically how the gradients are updated. Before we simple updadted it in the following way: 

p.data += -lr * 0.01 * p.grad. 

Now instead the optimizer keeps track of the gradient-history, such that it can create momentum in a certain direction and converge faster.

In [21]:
# Creating PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [22]:
# Training loop
batch_size = 32
for step in range(10000):
    
    # Sample a batch of data
    xb, yb = get_batch('train', batch_size, block_size)
    
    # Evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.515717029571533


In [23]:
# Generating some text
model_input = torch.zeros((1,1), dtype=torch.long) # Input token 0, which is \n
model_output = model.generate(model_input, max_new_tokens=300)[0].tolist()
print(f"Generated Text: \n {decode(model_output)}")

Generated Text: 
 
ADWent, athupy re un;
F he palisisZ.3Q&.
Se I gichey het s nt myon.

HENoon, feder n shil tIshe hel'stind l:
LLYe
Fis br b'theindy be worveat ar Jer's toshies sarlinderthordes dotcousthingur


TMeAu ifomefout aws irears hal vies than.
TI wir g hef winomougivane, I marus,
Anghe ale?

I coundn! yor be


# Exporting Code
Here we export all the code a script.

In [ ]:
!python3.10 ../../helpers/ipynb_to_py.py 4.\ Neural\ Network\ Module.ipynb "###NN###" ../../modules/Micrograd/nn.py

In [24]:
# CONTINUE: http://www.youtube.com/watch?v=kCc8FmEb1nY&t=38m11s